<a href="https://colab.research.google.com/github/raketic-ognjen/Market-Watch-Tech-Analysis/blob/main/TechAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = "DeepSkyBlue"><center>**Market Watch - Technical Analysis**</center></font>

### <center>**Core Idea Explanation**</center>

### Adding Packages - Libraries:
1. Pandas(pandas data reader)
2. yfinance
3. NumPy(linAlg)
4. SciPy(optimize and stats)
5. Plotly(graph objects and subplots)
6. Statsmodels.api
7. DateTime

In [1]:
from google.colab import files
uploaded0=files.upload()

Saving MCF_INV.py to MCF_INV.py


In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import numpy.linalg as LA
import scipy as sp
import scipy.optimize as sco
import scipy.stats as st
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import pandas_datareader as pdr
import datetime as dt
import MCF_INV as inv

> First of all, I want to appologise to my team and to You, professor, because I did not show up on the Market Watch Game due to my **TCL knee injury.** Because of that I will do next things:
1.   **Portfolio Optimization**
2.   **Efficient Frontiers**
3.   **CAPM Model**
4.   **Sharpe's Ratio**
5.   **Testing Portfolio Performances**
6.   **Robustness portfolio**
7.   **Black-Litterman model**
8.   **Comparing with Non-Technical Result of my group**



### <center>*Work*</center>

According to my group starting portfolio, we use 6 companies:
1. Starbucks - SBUX - 9 stocks 0.13 weight
2. Coca Cola - KO - 15  stocks 0.22 weight
3. Microsoft - MSFT 3 stocks 0.04 weight
4. Pfizer - PFE 25 stocks 0.37 weight
5. Nike - NKE 8 stocks 0.12 weight
6. Exxon Mobil - XOM 8 stocks 0.12 weight

Later Non Technical portfolio  had 83 trades and a lot buying-selling companies, so for this analysis I will use only this 6 stocks

According to TA starting portfolio, we use 6 companies:
1. Astra Zeneca - AZN -  
2. Nvidia - NVD -
3. Novartis - NVS
4. Rivian - RIVN
5. Carnival Corporation - CCL
6. Norwegian Cruise Company -NCLH

## NTA Portfolio

In [4]:
#Non Technical Analysis Portfolio start
ntaportf= ['SBUX', 'KO', 'MSFT','PFE', 'NKE', 'XOM']

### Preparation data for optimization of NTA portfolio

In [5]:
ntaprices = yf.download(ntaportf,start='2022-05-18',end='2023-05-18')['Adj Close']
#ntaprices.to_csv('ntaprices.csv',encoding='utf-8-sig')
#files.download('ntaprices.csv')

[*********************100%%**********************]  6 of 6 completed


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
rets1=pd.read_csv('ntaprices.csv',index_col=0)

In [7]:
#returns on NTA portfolio
rets1=ntaprices.pct_change().dropna()

#Weights for NTA
w1 = [0.13,0.22,0.04,0.37,0.12,0.12]

#Portfolio returns
rp1=rets1@w1
rp1

Date
2022-05-19   -0.000856
2022-05-20    0.018884
2022-05-23    0.016880
2022-05-24    0.004252
2022-05-25    0.006996
                ...   
2023-05-11   -0.007343
2023-05-12   -0.002677
2023-05-15   -0.001188
2023-05-16   -0.006898
2023-05-17    0.002667
Length: 250, dtype: float64

### From portfolio returns (Portfolio as an asset)

In [8]:
inv.expRet(rets1,True),inv.annualize_vol(rets1,True)

(KO      0.000296
 MSFT    0.001101
 NKE     0.000597
 PFE    -0.001012
 SBUX    0.001834
 XOM     0.000927
 dtype: float64,
 KO      0.010386
 MSFT    0.020836
 NKE     0.023806
 PFE     0.014571
 SBUX    0.018550
 XOM     0.020540
 dtype: float64)

In [9]:
er1= inv.expRet(rp1)
print(er1)
vol1= inv.annualize_vol(rp1)
print(vol1)

0.06586676303885548
0.19451632729201515


### From stocks' returns (Portfolio as collection of assets)

In [10]:
per1 = inv.per(W=w1,ret=rets1, n=250)
per1

0.0653440109512455

Computation risk of Portfolio

In [11]:
pV1=inv.pV(w1,rets1, 250, True)
pV1

0.19374289946839085

### MVP

In [12]:
mvp1= inv.mvp(ret=rets1,f=250,com=True)
mvp1

{'w': array([ 0.74264398, -0.0152481 , -0.08163823,  0.15621428,  0.07963317,
         0.11839491]),
 'er': 0.06202074661378879,
 'vol': 0.15366168738953895}

In [13]:
mvp1['w']

array([ 0.74264398, -0.0152481 , -0.08163823,  0.15621428,  0.07963317,
        0.11839491])

### Manual optimization

In [14]:
res1=sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [0.13,0.22,0.04,0.37,0.12,0.12],
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1)])
res1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.023800713595941703
       x: [ 7.433e-01 -1.615e-02 -8.157e-02  1.548e-01  8.104e-02
            1.185e-01]
     nit: 15
     jac: [ 4.757e-02  4.712e-02  4.748e-02  4.758e-02  4.759e-02
            4.769e-02]
    nfev: 106
    njev: 15

In [15]:
target1= inv.targetP(rets1,0.1)
target1

{'w': array([ 0.75048359, -0.00959368, -0.0921333 ,  0.10127585,  0.1224916 ,
         0.12747595]),
 'er': 0.1,
 'vol': 0.15487295732670778}

In [16]:
result1 = sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [0.13,0.22,0.04,0.37,0.12,0.12],
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1),
                               dict(type='eq',fun=lambda w1: inv.per(w1,rets1)-0.08)])
result1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.023837385911253083
       x: [ 7.449e-01 -1.489e-02 -8.362e-02  1.311e-01  9.893e-02
            1.236e-01]
     nit: 11
     jac: [ 4.764e-02  4.821e-02  4.875e-02  4.618e-02  4.970e-02
            4.866e-02]
    nfev: 78
    njev: 11

In [17]:
resnta = sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [1/6]*6,
                      bounds=[[0,0.04]]*6,
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1),
                               dict(type='eq',fun=lambda w1: inv.per(w1,rets1)-0.1)])
resnta

 message: Iteration limit reached
 success: False
  status: 9
     fun: 0.0024526425141712374
       x: [ 4.000e-02  4.000e-02  4.000e-02  4.000e-02  4.000e-02
            4.000e-02]
     nit: 100
     jac: [ 1.083e-02  2.525e-02  3.086e-02  1.440e-02  2.246e-02
            1.884e-02]
    nfev: 1489
    njev: 100

### Efficient Frontier for NTA portfolio

In [84]:
ntaEF = inv.EF(rets1,plot='full')
ntaEF

In [85]:
fignta=inv.EF(rets1,Range=[0.01,1.5],plot='full')
fignta

In [88]:
rfa1 = pdr.get_data_fred('DTB3',start='2022-05-18',end='2023-05-18')/100
rfa1.head()
rfa1.dropna()

,DTB3
DATE,
2022-05-18,0.0102
2022-05-19,0.0104
2022-05-20,0.0101
2022-05-23,0.0107
2022-05-24,0.0105
...,...
2023-05-12,0.0508
2023-05-15,0.0506
2023-05-16,0.0507


### Deannualization(EAR)

In [92]:
rf1 = (1+rfa1)**(1/251)-1
rf1.head()
rf1.dropna()

,DTB3
DATE,
2022-05-18,0.000040
2022-05-19,0.000041
2022-05-20,0.000040
2022-05-23,0.000042
2022-05-24,0.000042
...,...
2023-05-12,0.000197
2023-05-15,0.000197
2023-05-16,0.000197


In [98]:
opt1 = inv.maxSharpe(rets1,rf1,f=251,com=True)
opt1

{'w': array([ 0.8861064 ,  0.27905347, -0.46499754, -1.        ,  1.        ,
         0.29983767]),
 'er': 0.908078510382051,
 'vol': 0.341156186978444,
 'sharpe': 2.5565581825879486}

In [107]:
fignta.add_trace(go.Scatter(x=[inv.pV(opt1['w'],rets1,n=250,vol=True)],
                            y=[inv.per(opt1['w'],rets1)],
                            marker=dict(
                                color='Crimson',
                                size=10,
                                line=dict(
                                    color='Black',
                                    width=2
                                )
                            )
                          )
)

#plot show
fignta.show()

Test for Eff frontier

In [109]:
#Initial Wealth
Wealth1 = 30_000

#Determined amount invested in each stock
si1=opt1['w']*Wealth1

#Number of stocks
n1=si1/ntaprices.iloc[0]
n1

KO      450.629706
MSFT     33.337616
NKE    -129.850323
PFE    -624.034854
SBUX    432.262392
XOM     103.501854
Name: 2022-05-18 00:00:00, dtype: float64

In [110]:
#Value of portfolio each point in the time
ST1 = ntaprices@n1
ST1

Date
2022-05-18    30000.000000
2022-05-19    29994.884933
2022-05-20    29805.070300
2022-05-23    30777.892149
2022-05-24    30859.249587
                  ...     
2023-05-11    56080.180742
2023-05-12    56765.040825
2023-05-15    57115.665808
2023-05-16    56547.503863
2023-05-17    57122.376671
Length: 251, dtype: float64

### Static Trading Strategy for NTA portfolio

In [129]:
#preparing data
return1 = ntaprices.iloc[:,:6].pct_change()
return1.dropna(inplace=True)

#initial wealth
v=30_000

#Optimal weights
ntaMVP1 = inv.mvp(return1,f=250)['w']
nta15= inv.targetP(return1,0.15)['w']

In [130]:
#optimal weights for the MVP NTA Portfolio
ntaMVP1

array([ 0.74264398, -0.0152481 , -0.08163823,  0.15621428,  0.07963317,
        0.11839491])

In [131]:
#Optimal weights for portfolio with the target expected return of 15 percent per annum
nta15

array([ 0.76247644,  0.01082   , -0.11340892,  0.02548814,  0.17586967,
        0.13875467])

In [134]:
#Vector of money invested into each asset
vntaMVP1 = ntaMVP1*v
vnta15= nta15*v

In [135]:
vntaMVP1

array([22279.31933071,  -457.44301941, -2449.14704846,  4686.42840844,
        2388.99511392,  3551.8472148 ])

In [136]:
vnta15

array([22874.29318368,   324.60003764, -3402.26767963,   764.64421978,
        5276.09002334,  4162.64021519])

In [137]:
#vector of innitial stock prices
ntap0 = ntaprices.iloc[0,:6]
ntap0

KO       58.991211
MSFT    251.115860
NKE     107.430817
PFE      48.074238
SBUX     69.402290
XOM      86.907913
Name: 2022-05-18 00:00:00, dtype: float64

In [144]:
#Number of shares
nMVPnta1= vntaMVP1/ntap0
n15nta1 = vnta15/ntap0

In [140]:
#number of stocks for MVP NTA is
nMVPnta1

KO      377.671843
MSFT     -1.821641
NKE     -22.797435
PFE      97.483156
SBUX     34.422425
XOM      40.869089
Name: 2022-05-18 00:00:00, dtype: float64

In [145]:
#number of stocks for portfolio with target expected return of 15% annually
n15nta1

KO      387.757648
MSFT      1.292631
NKE     -31.669383
PFE      15.905488
SBUX     76.021843
XOM      47.897137
Name: 2022-05-18 00:00:00, dtype: float64

In [148]:
#the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two
VntaMVP = ntaprices.iloc[:,:6]@nMVPnta1
V15nta = ntaprices.iloc[:,:6]@n15nta1

### Graphical Comparation of Static Strategy for NTA portfolio

In [150]:
fignta1 = go.Figure()
fignta1.add_trace(
    go.Scatter(x=ntaprices.index,
               y=VntaMVP,
               line=dict(color='Blue',width=3),
               name='MVP NTA portfolio')
)
fignta1.add_trace(
    go.Scatter(
        x=ntaprices.index,
        y=V15nta,
        line=dict(color='Crimson',width=3),
        name='Target NTA 15% pa')
)
fignta1.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing 2 Static Strategy for NTA portfolio',
               x = 0.5,
               y = 0.87,
               font=dict(size=25,color='Navy'))
)

fignta1.show()

In [151]:
V15nta[-1]-VntaMVP[-1]

2716.0085428953753

15% NTA Target porfolio outperform MVP NTA portfolio for 9.05%

In [152]:
ntaprices.iloc[-1,:6]

KO       62.672691
MSFT    313.336609
NKE     116.206444
PFE      36.345070
SBUX    105.129402
XOM     103.967728
Name: 2023-05-17 00:00:00, dtype: float64

In [153]:
#Weights on the last day of the trade
nMVPnta1*ntaprices.iloc[-1,:6]/VntaMVP[-1]

KO      0.742914
MSFT   -0.017915
NKE    -0.083150
PFE     0.111204
SBUX    0.113582
XOM     0.133364
dtype: float64

In [154]:
#initial Weights
ntaMVP1

array([ 0.74264398, -0.0152481 , -0.08163823,  0.15621428,  0.07963317,
        0.11839491])

# TA Portfolio

### Data Preparation

In [18]:
#Technical Analysis portfolio start
taportf = ['AZN', 'NVDA', 'NVS', 'RIVN', 'CCL', 'NCLH']

In [19]:
taprices = yf.download(taportf,start='2022-05-18',end='2023-05-18')['Adj Close']
taprices.to_csv('taprices.csv',encoding='utf-8-sig')
files.download('taprices.csv')

[*********************100%%**********************]  6 of 6 completed


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
rets2= pd.read_csv('taprices.csv',index_col=0)

### From portfolio returns (Portfolio as an asset)

In [52]:
#returns
rets2= taprices.pct_change().dropna()
rets2

#Weights
w2=[0.13,0.22,0.04,0.37,0.12,0.12]

#return on portfolio
rp2 = rets2@w2
rp2

Date
2022-05-19    0.014090
2022-05-20   -0.016602
2022-05-23    0.001928
2022-05-24   -0.047916
2022-05-25    0.030667
                ...   
2023-05-11   -0.013151
2023-05-12   -0.021508
2023-05-15    0.025521
2023-05-16    0.000646
2023-05-17    0.029414
Length: 250, dtype: float64

In [53]:
inv.expRet(rets2,250,True),inv.annualize_vol(rets2,250)

(AZN     0.205298
 CCL    -0.202312
 NCLH   -0.097484
 NVDA    0.783339
 NVS     0.204596
 RIVN   -0.508611
 dtype: float64,
 AZN     0.235327
 CCL     0.737278
 NCLH    0.705900
 NVDA    0.555384
 NVS     0.196501
 RIVN    0.731755
 dtype: float64)

In [54]:
er2 = inv.expRet(rp2)
print(er2)
vol2= inv.annualize_vol(rp2)
print(vol2)

0.28924429356366166
0.4250338474874931


### From stocks' returns (Portfolio as collection of assets)

In [55]:
#Expected Return on portfolio
per2=inv.per(w2,rets2,250,True)
per2

0.23163447480059346

In [56]:
pV2 = inv.pV(w2,rets2,250,True)
pV2

0.42334384537710257

###MVP

In [57]:
mvp2 = inv.mvp(rets2,250,True)
mvp2

{'w': array([ 0.24931125, -0.05201149,  0.05462122,  0.02012366,  0.69997414,
         0.02798123]),
 'er': 0.2011250794062859,
 'vol': 0.1861652370737557}

### Manual optimization

In [59]:
res2 = sco.minimize(lambda w2: pV2,
                        [0.13,0.22,0.04,0.37,0.12,0.12],
                       constraints=[dict(type='eq',fun=lambda w2:sum(w2)-1)])
res2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.42334384537710257
       x: [ 1.300e-01  2.200e-01  4.000e-02  3.700e-01  1.200e-01
            1.200e-01]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 7
    njev: 1

Expected return is 42.34%

In [60]:
target2=inv.targetP(rets2,0.45)
target2

{'w': array([ 0.19373596, -0.10761355,  0.0628258 ,  0.27633805,  0.69960978,
        -0.12489604]),
 'er': 0.45,
 'vol': 0.2217006421827916}

In [63]:
result2 = sco.minimize(lambda w2: pV2,
                       [0.13,0.22,0.04,0.37,0.12,0.12],
                       constraints=[dict(type='eq',fun=lambda w2: sum(w2)-1),
                                    dict(type='eq',fun=lambda w2: inv.per(w2,rets2,250,True)-0.45)])
result2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.42334384537710257
       x: [ 1.614e-01  1.608e-01  4.063e-03  5.299e-01  1.512e-01
           -7.353e-03]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 8
    njev: 1

In [81]:
tares2 = sco.minimize(lambda w2: pV2,
                       [0.13,0.22,0.04,0.37,0.12,0.12],
                        bounds=[[0,0.39]]*6,
                       constraints=[dict(type='eq',fun=lambda w2: sum(w2)-1),
                                    dict(type='eq',fun=lambda w2: inv.per(w2,rets2)-0.4)])
tares2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.42334384537710257
       x: [ 2.491e-01  8.294e-02  5.301e-02  3.900e-01  2.250e-01
            0.000e+00]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 8
    njev: 1

In [82]:
tares2.x

array([0.24907114, 0.08294405, 0.05300783, 0.39      , 0.22497698,
       0.        ])

### Efficient Frontier for TA poftolio

In [111]:
taEF = inv.EF(rets2,plot='full')
taEF

In [113]:
figta=inv.EF(rets2,Range=[0.01,1.5],plot='full')
figta

In [114]:
rfa2 = pdr.get_data_fred('DTB3',start='2022-05-18',end='2023-05-18')/100
rfa2.head()
rfa2.dropna()

,DTB3
DATE,
2022-05-18,0.0102
2022-05-19,0.0104
2022-05-20,0.0101
2022-05-23,0.0107
2022-05-24,0.0105
...,...
2023-05-12,0.0508
2023-05-15,0.0506
2023-05-16,0.0507


In [115]:
rf2 = (1+rfa2)**(1/251)-1
rf2.head()
rf2.dropna()

,DTB3
DATE,
2022-05-18,0.000040
2022-05-19,0.000041
2022-05-20,0.000040
2022-05-23,0.000042
2022-05-24,0.000042
...,...
2023-05-12,0.000197
2023-05-15,0.000197
2023-05-16,0.000197


In [116]:
opt2 = inv.maxSharpe(rets2,rf2,f=251,com=True)
opt2

{'w': array([ 0.04405029, -0.31063078,  0.0048511 ,  1.        ,  0.7443996 ,
        -0.4826702 ]),
 'er': 1.258281318496821,
 'vol': 0.481653929039205,
 'sharpe': 2.5378978054085737}

In [118]:
figta.add_trace(go.Scatter(x=[inv.pV(opt2['w'],rets2,n=250,vol=True)],
                            y=[inv.per(opt2['w'],rets2)],
                            marker=dict(
                                color='Crimson',
                                size=10,
                                line=dict(
                                    color='Black',
                                    width=2
                                )
                            )
                          )
)

#plot show
figta.show()

### Test for Eff Frontier in TA Portfolio

In [119]:
#Initial Wealth
Wealth2 = 30_000

#Determined amount invested in each stock
si2=opt2['w']*Wealth2

#Number of stocks
n2=si2/taprices.iloc[0]
n2

AZN      21.462798
CCL    -673.332612
NCLH      9.153012
NVDA    177.315026
NVS     265.767903
RIVN   -530.601159
Name: 2022-05-18 00:00:00, dtype: float64

In [120]:
#Value of portfolio each point in the time
ST2 = taprices@n2
ST2

Date
2022-05-18    30000.000000
2022-05-19    29546.488500
2022-05-20    29679.022880
2022-05-23    30878.173213
2022-05-24    31543.274197
                  ...     
2023-05-11    65629.424418
2023-05-12    65898.756101
2023-05-15    66385.605134
2023-05-16    66925.583229
2023-05-17    67576.414052
Length: 251, dtype: float64

# Graphical Comparation of Effective Frontier

In [127]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=ST1.index,y= ST1, name = 'NTA Portfolio'))
fig.add_trace(go.Scatter(x=ST2.index,y=ST2, name = 'TA Portfolio'))


fig.show()